In [90]:
import pandas as pd
import numpy as np
import plotly.express as px

In [116]:
ORCHESTRATORS_ORDER = ["AIRFLOW", "ARGO WORKFLOW", "WINDMILL", "TEMPORAL"]

EXPERIMENT = "scenar_1_1_"

METRICS_LIST = [
    "cpu_mean",
    "cpu_max",
    "memory_mean",
    "memory_max",
    "sched_time_mean",
    "sched_time_max",
    "exec_time_mean",
    "exec_time_max"]

In [117]:
def plot_metric(df: pd.DataFrame, metric: str) -> None:
    """
    Plot the specified metric for different orchestrators using Plotly.
    """

    # Keep only CPU metrics for now
    df_red = df[df["experiment"].str.contains(metric)].copy()



    if len(df_red) == 0:
        print(f"No data for metric: {metric}")
        return
    

    # Long format: orchestrator / value
    long = df_red.melt(
        id_vars="experiment",
        var_name="orchestrator",
        value_name="value"
    )

    long['value'].clip(lower=1e-4, inplace=True)

    fig = px.bar(
        long,
        x="orchestrator",
        y="value",
        color="experiment",
        title=f"{metric} usage by orchestrator",
        color_discrete_sequence=["#601ef9", "#658354"]
    )

    fig.update_layout(
        barmode="stack",
        yaxis_title=f"{metric} during 1 hour",
        font=dict(size=16),
        legend=dict(
            title="Component",
            orientation="v",
            x=1.02,
            y=0.5,
            font=dict(size=18),
        ),
        xaxis_title=""
    )

    if long['value'].max() / long['value'].min() > 20:
        fig.update_yaxes(type="log")
        fig.update_layout(title=f"{metric} usage by orchestrator - LOG SCALE")


    fig.show()


In [119]:
def experiment_parsing(df: pd.DataFrame, experiment: str) -> pd.DataFrame:
    """
    Return rows where the 'experiment' column starts with the provided experiment string.
    Handles missing values safely and returns a copy with reset index.
    """
    df_res = df.copy()
    if not isinstance(experiment, str) or experiment == "":
        return df_res.copy()

    mask = df_res["experiment"].fillna("").astype(str).str.startswith(experiment)

    df_red = df_res.loc[mask].reset_index(drop=True)

    df_red['experiment'] = df_red['experiment'].apply(lambda x: x.split(experiment)[1])

    df_red = df_red[['experiment'] + ORCHESTRATORS_ORDER]

    return df_red

--------------

In [120]:
df_results = pd.read_csv("result/bench_raw_results.csv")
df_results.rename(columns={"Unnamed: 0": "experiment"}, inplace=True)
df_re = experiment_parsing(df_results, EXPERIMENT)
df_re

,experiment,AIRFLOW,ARGO WORKFLOW,WINDMILL,TEMPORAL
0,perm_cpu_mean,0.97,0.010,0.42,0.880
1,perm_cpu_max,1.34,0.024,0.46,1.030
2,perm_memory_mean,4436.00,128.000,2536.00,1490.000
3,perm_memory_max,4755.00,96.000,2550.00,1505.000
4,work_cpu_mean,0.53,0.060,0.47,0.188
5,work_cpu_max,1.66,0.800,1.64,0.980
6,work_memory_mean,3933.00,97.000,523.00,384.000
7,work_memory_max,5067.00,181.000,580.00,414.000
8,sched_time_mean,3.64,4.740,1.05,1.530
9,sched_time_max,4.58,4.970,1.15,1.540


In [121]:
for metric in METRICS_LIST:
    plot_metric(df_re, metric)
    print("-"*40)

----------------------------------------


----------------------------------------


----------------------------------------


----------------------------------------


----------------------------------------


----------------------------------------


----------------------------------------


----------------------------------------
